<a href="https://colab.research.google.com/github/trhieu2/hands-on-ml-practices/blob/main/cafa6_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# TẠO MỘT ĐƯỜNG DẪN AN TOÀN TRÊN DRIVE CỦA BẠN
# (Bạn có thể tạo thư mục CAFA6_Project trong MyDrive trước)
SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs(SAVE_PATH, exist_ok=True)

print(f"Các file sẽ được lưu tại: {SAVE_PATH}")

Các file sẽ được lưu tại: /content/drive/MyDrive/CAFA6_Project/


In [3]:
import numpy as np
import pandas as pd

In [16]:
!pip install obonet

In [17]:
import obonet
import networkx as nx

In [34]:
graph = obonet.read_obo('/content/drive/MyDrive/cafa6-data/Train/go-basic.obo')

nodes_data = []
for node_id, data in graph.nodes(data=True):
    node_info = {'id': node_id}
    node_info.update(data)
    nodes_data.append(node_info)

nodes_df = pd.DataFrame(nodes_data)

In [ ]:
nodes_df.head()

,id,name,namespace,def,synonym,is_a,xref,alt_id,subset,relationship,comment
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","[""mitochondrial inheritance"" EXACT []]","[GO:0048308, GO:0048311]",NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,[GO:0007005],NaN,NaN,NaN,NaN,NaN
2,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...","[""high affinity zinc uptake transmembrane tran...",[GO:0005385],NaN,NaN,NaN,NaN,NaN
3,GO:0000007,low-affinity zinc ion transmembrane transporte...,molecular_function,"""Enables the transfer of a solute or solutes f...",NaN,[GO:0005385],NaN,NaN,NaN,NaN,NaN
4,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,"""Catalysis of the transfer of a mannose residu...","[""1,6-alpha-mannosyltransferase activity"" EXAC...",[GO:0000030],"[Reactome:R-HSA-449718 ""Addition of a third ma...",NaN,NaN,NaN,NaN


In [45]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.8 MB/s eta 0:00:00


In [46]:
from Bio import SeqIO

taxonomy_df = pd.read_csv('/content/drive/MyDrive/cafa6-data/Train/train_taxonomy.tsv', sep='\t', names=['EntryID', 'taxon ID'])
print(taxonomy_df.shape)

(82404, 2)


In [ ]:
taxonomy_df.head()

,EntryID,taxon ID
0,A0A0C5B5G6,9606
1,A0JNW5,9606
2,A0JP26,9606
3,A0PK11,9606
4,A1A4S6,9606


In [47]:
sequences = list(SeqIO.parse('/content/drive/MyDrive/cafa6-data/Train/train_sequences.fasta', 'fasta'))
sequences[:1]

[SeqRecord(seq=Seq('MRWQEMGYIFYPRKLR'), id='sp|A0A0C5B5G6|MOTSC_HUMAN', name='sp|A0A0C5B5G6|MOTSC_HUMAN', description='sp|A0A0C5B5G6|MOTSC_HUMAN Mitochondrial-derived peptide MOTS-c OS=Homo sapiens OX=9606 GN=MT-RNR1 PE=1 SV=1', dbxrefs=[])]

In [48]:
sequences_data = []
for seq in sequences:
    sequences_data.append({
        "EntryID": seq.id.split('|')[1],
        "Gene Name": seq.id.split('|')[2],
        "Amino-acid sequence": str(seq.seq),
        "Sequence length": len(seq.seq)
    })

sequences_df = pd.DataFrame(sequences_data)

sequences_df.head()

,EntryID,Gene Name,Amino-acid sequence,Sequence length
0,A0A0C5B5G6,MOTSC_HUMAN,MRWQEMGYIFYPRKLR,16
1,A0JNW5,BLT3B_HUMAN,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,1464
2,A0JP26,POTB3_HUMAN,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,581
3,A0PK11,CLRN2_HUMAN,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,232
4,A1A4S6,RHG10_HUMAN,MGLQPLEFSDCYLDSPWFRERIRAHEAELERTNKFIKELIKDGKNL...,786


In [41]:
terms_df = pd.read_csv('/content/drive/MyDrive/cafa6-data/Train/train_terms.tsv', sep='\t')
print(terms_df.shape)

(537027, 3)


In [ ]:
terms_df.head()

,EntryID,term,aspect
0,Q5W0B1,GO:0000785,C
1,Q5W0B1,GO:0004842,F
2,Q5W0B1,GO:0051865,P
3,Q5W0B1,GO:0006275,P
4,Q5W0B1,GO:0006513,P


## Setup

In [4]:
!pip install transformers accelerate

In [5]:
import torch
from transformers import EsmTokenizer, EsmModel

model_name = "facebook/esm2_t30_150M_UR50D"
tokenizer = EsmTokenizer.from_pretrained(model_name)

model = EsmModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
print(f"Model loaded on: {device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on: cuda


## Create embeddings

In [14]:
from tqdm.auto import tqdm

def get_embeddings(sequence_batch):
    """
    Hàm này nhận vào một danh sách các chuỗi amino acid
    và trả về một numpy array chứa các embeddings.
    """
    # 1. Tokenize: Chuyển chuỗi thành các token (số) mà mô hình hiểu
    # padding=True: Thêm token "đệm" để các chuỗi trong batch dài bằng nhau
    # truncation=True: Cắt bớt các chuỗi quá dài (ESM có giới hạn)
    # max_length=1024: Một giới hạn an toàn cho độ dài chuỗi
    # return_tensors="pt": Trả về dưới dạng PyTorch Tensors
    inputs = tokenizer(
        sequence_batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    )

    # 2. Chuyển dữ liệu input sang GPU
    # Đây là bước quan trọng để tăng tốc!
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 3. Đưa qua mô hình
    # torch.no_grad() tắt việc tính toán đạo hàm (gradient),
    # giúp tiết kiệm bộ nhớ và tăng tốc vì chúng ta chỉ "dự đoán" (inference)
    with torch.no_grad():
        outputs = model(**inputs)

    # 4. Lấy embeddings
    # outputs.last_hidden_state có shape: (batch_size, sequence_length, embedding_dim)
    # Chúng ta lấy trung bình (mean) theo chiều sequence_length (dim=1)
    # để tạo ra 1 vector duy nhất cho mỗi protein
    # Shape cuối: (batch_size, embedding_dim)
    embeddings = outputs.last_hidden_state.mean(dim=1)

    # 5. Chuyển embeddings về CPU và sang dạng numpy để xử lý tiếp
    return embeddings.cpu().numpy()

### Test on small batch

In [ ]:
# Lấy 5 chuỗi đầu tiên từ DataFrame bạn đã tải
test_sequences = sequences_df['Amino-acid sequence'].head(5).tolist()

print("Bắt đầu tạo embedding cho 5 chuỗi thử nghiệm...")
test_embeddings = get_embeddings(test_sequences)

print("Hoàn thành!")
print(f"Shape của embeddings (Số lượng, Chiều vector): {test_embeddings.shape}")

Bắt đầu tạo embedding cho 5 chuỗi thử nghiệm...
Hoàn thành!
Shape của embeddings (Số lượng, Chiều vector): (5, 640)


### Run on riel set

In [ ]:
# Kích thước batch, bạn có thể thử tăng lên 128 nếu GPU cho phép
BATCH_SIZE = 64

# Danh sách để lưu trữ các batch embeddings
all_embeddings = []

# Lấy toàn bộ danh sách chuỗi
sequences = sequences_df['Amino-acid sequence'].tolist()

print(f"Bắt đầu tạo embeddings cho {len(sequences)} proteins...")
print(f"Tổng số batch: { (len(sequences) // BATCH_SIZE) + 1 }")

# Vòng lặp chính, đi qua danh sách theo từng BATCH_SIZE
# tqdm sẽ cho chúng ta một thanh tiến trình đẹp mắt
for i in tqdm(range(0, len(sequences), BATCH_SIZE)):
    # Lấy ra 1 batch
    batch_seqs = sequences[i:i + BATCH_SIZE]

    # Tạo embeddings cho batch đó
    batch_embeds = get_embeddings(batch_seqs)

    # Thêm vào danh sách
    all_embeddings.append(batch_embeds)

print("Đã tạo xong embeddings cho tất cả protein!")

# Ghép tất cả các batch lại thành một ma trận numpy lớn
full_embeddings = np.concatenate(all_embeddings, axis=0)

print(f"Shape của ma trận embeddings cuối cùng: {full_embeddings.shape}")

Bắt đầu tạo embeddings cho 82404 proteins...
Tổng số batch: 1288


  0%|          | 0/1288 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Save embeddings



In [ ]:
np.save(SAVE_PATH + 'train_embeddings_esm150M.npy', full_embeddings)

print("Đã lưu embeddings ra file 'train_embeddings_esm150M.npy'")

## Prepare labels

### Get GO terms list

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
import joblib # Dùng để lưu binarizer

In [ ]:
all_terms = sorted(list(terms_df['term'].unique()))
num_classes = len(all_terms)

print(f"Tổng số lớp (GO term) duy nhất: {num_classes}")

### Group labels

In [ ]:
labels_grouped_df = terms_df.groupby('EntryID')['term'].apply(list).reset_index()
ordered_entry_ids_df = sequences_df[['EntryID']]
aligned_labels_df = ordered_entry_ids_df.merge(labels_grouped_df, on='EntryID', how='left')
aligned_labels_df['term'] = aligned_labels_df['term'].apply(lambda x: x if isinstance(x, list) else [])

print("DataFrame labels đã căn chỉnh (5 hàng đầu):")
print(aligned_labels_df.head())
print(f"Shape của DataFrame labels đã căn chỉnh: {aligned_labels_df.shape}")

### Binarize and store labels

In [ ]:
mlb = MultiLabelBinarizer(classes=all_terms)
labels_list = aligned_labels_df['term'].tolist()
full_labels_matrix = mlb.fit_transform(labels_list)

print(f"Shape Embeddings (X): {full_embeddings.shape}")
print(f"Shape Labels (y):     {full_labels_matrix.shape}")

In [ ]:
np.save(SAVE_PATH + 'train_labels.npy', full_labels_matrix)
print("Đã lưu ma trận labels (y) ra file 'train_labels.npy'")

joblib.dump(mlb, SAVE_PATH + 'multilabel_binarizer.joblib')
print("Đã lưu MultiLabelBinarizer (mlb) ra file 'multilabel_binarizer.joblib'")

## Model training

In [7]:
!pip install -q tensorflow

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

X = np.load('/content/drive/MyDrive/CAFA6_Project/train_embeddings_esm150M.npy')
y = np.load('/content/drive/MyDrive/CAFA6_Project/train_labels.npy')

print(f"Shape của X: {X.shape}")
print(f"Shape của y: {y.shape}")

Shape của X: (82404, 640)
Shape của y: (82404, 26125)


### Train-Test split

In [21]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.1,
    random_state=42
)

### Construct a neural network

Input (640) -> Dense(1024) -> Dense(1024) -> Output (số_lớp)

In [22]:
embedding_dim = X_train.shape[1]
num_classes = y_train.shape[1]

print(f"Input dimension (Embedding): {embedding_dim}")
print(f"Output dimension (Classes): {num_classes}")

# 1. Định nghĩa Input Layer
inputs = Input(shape=(embedding_dim,))

# 2. Xây dựng các lớp ẩn (Hidden Layers)
# Lớp 1
x = Dense(1024, activation='relu')(inputs)
x = BatchNormalization()(x) # Giúp ổn định quá trình học
x = Dropout(0.3)(x)         # Chống overfitting (học vẹt)

# Lớp 2
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# 3. Lớp Output (Đầu ra)
# Dùng activation 'sigmoid' vì đây là bài toán multi-label
# Mỗi neuron output sẽ cho ra xác suất (0 đến 1) cho 1 GO term
outputs = Dense(num_classes, activation='sigmoid')(x)

# 4. Tạo mô hình
model = Model(inputs, outputs)

Input dimension (Embedding): 640
Output dimension (Classes): 26125


### Compile

In [23]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=BinaryCrossentropy()
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       656,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 26125)          │    26,778,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,492,301 (108.69 MB)

 Trainable params: 28,488,205 (108.67 MB)

 Non-trainable params: 4,096 (16.00 KB)

### Training


In [24]:
checkpoint = ModelCheckpoint(
    SAVE_PATH + 'best_model.keras',        # Tên file để lưu mô hình
    monitor='val_loss',     # Theo dõi val_loss
    save_best_only=True,    # Chỉ lưu mô hình tốt nhất
    mode='min',             # Chế độ 'min' vì chúng ta muốn val_loss thấp nhất
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,             # Dừng nếu val_loss không cải thiện sau 3 epochs
    mode='min',
    verbose=1
)

In [25]:
BATCH_SIZE = 256
EPOCHS = 20 # Bắt đầu với 20, EarlyStopping có thể sẽ dừng sớm hơn

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.3596
Epoch 1: val_loss improved from inf to 0.00326, saving model to /content/drive/MyDrive/CAFA6_Project/best_model.keras
290/290 ━━━━━━━━━━━━━━━━━━━━ 26s 76ms/step - loss: 0.3588 - val_loss: 0.0033
Epoch 2/20
289/290 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0033
Epoch 2: val_loss improved from 0.00326 to 0.00193, saving model to /content/drive/MyDrive/CAFA6_Project/best_model.keras
290/290 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.0033 - val_loss: 0.0019
Epoch 3/20
289/290 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0021
Epoch 3: val_loss improved from 0.00193 to 0.00171, saving model to /content/drive/MyDrive/CAFA6_Project/best_model.keras
290/290 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.0021 - val_loss: 0.0017
Epoch 4/20
288/290 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0018
Epoch 4: val_loss improved from 0.00171 to 0.00162, saving model to /content/drive/MyDrive/CAFA6_Project/best_model.keras
290/290 ━━━━━━━

## Submission file

In [26]:
# Hàm đọc file FASTA đơn giản
def read_fasta(file_path):
    sequences = []
    entry_ids = []
    current_seq = ""
    current_id = ""

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                # Nếu có chuỗi cũ, lưu lại
                if current_id:
                    sequences.append(current_seq)
                    entry_ids.append(current_id)

                # Bắt đầu chuỗi mới
                current_id = line[1:].split()[0] # Lấy EntryID
                current_seq = ""
            else:
                current_seq += line

        # Đừng quên lưu chuỗi cuối cùng
        if current_id:
            sequences.append(current_seq)
            entry_ids.append(current_id)

    return pd.DataFrame({'EntryID': entry_ids, 'Amino-acid sequence': sequences})

print("Đang đọc file test_sequences.fasta...")
# Thay thế bằng đường dẫn đúng nếu cần
test_sequences_df = read_fasta('/content/drive/MyDrive/cafa6-data/Test/testsuperset.fasta')

print(f"Đã tải {len(test_sequences_df)} chuỗi protein từ tập test.")
print(test_sequences_df.head())

Đang đọc file test_sequences.fasta...
Đã tải 224309 chuỗi protein từ tập test.
      EntryID                                Amino-acid sequence
0  A0A0C5B5G6                                   MRWQEMGYIFYPRKLR
1  A0A1B0GTW7  MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...
2      A0JNW5  MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...
3      A0JP26  MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...
4      A0PK11  MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...


In [ ]:
import os
import glob
import numpy as np
from tqdm.auto import tqdm

# --- ĐẢM BẢO CÁC BIẾN NÀY ĐÃ TỒN TẠI ---
# 1. SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'
# 2. test_sequences_df (đã đọc từ file fasta)
# 3. Hàm get_embeddings()
# 4. Biến 'model' là MÔ HÌNH ESM-2 (đã tải lại từ Bước 1)
# ----------------------------------------------------

BATCH_SIZE = 64 # Bạn có thể dùng 128 nếu GPU mạnh

# Tạo một thư mục trên DRIVE của bạn để lưu các batch
output_dir = os.path.join(SAVE_PATH, 'test_embeddings_batches')
os.makedirs(output_dir, exist_ok=True)

print(f"Thư mục lưu batch test: {output_dir}")

# Lấy danh sách chuỗi
test_sequences = test_sequences_df['Amino-acid sequence'].tolist()
print(f"Bắt đầu tạo embeddings cho {len(test_sequences)} protein test...")

# --- VÒNG LẶP AN TOÀN ---
for i in tqdm(range(0, len(test_sequences), BATCH_SIZE), desc="Tạo test embedding"):

    batch_index = i // BATCH_SIZE
    # File path trỏ thẳng vào Google Drive
    file_path = os.path.join(output_dir, f'batch_{batch_index:04d}.npy')

    # *** PHẦN SỬA QUAN TRỌNG: BỎ QUA NẾU ĐÃ LƯU ***
    # Nếu file batch này đã tồn tại, bỏ qua và sang vòng lặp tiếp theo
    if os.path.exists(file_path):
        continue

    # Lấy batch và tạo embedding
    batch_seqs = test_sequences[i:i + BATCH_SIZE]
    batch_embeds = get_embeddings(batch_seqs)

    # *** PHẦN SỬA QUAN TRỌNG: LƯU BATCH NGAY LẬP TỨC ***
    # Lưu ngay batch này vào Google Drive
    np.save(file_path, batch_embeds)

print("Đã hoàn thành tạo tất cả các batch embeddings!")

# --- GHÉP TẤT CẢ CÁC FILE BATCH LẠI ---
print("Đang ghép các file batch từ Google Drive...")

# Tìm tất cả các file .npy trong thư mục batch
# Sắp xếp lại để đảm bảo đúng thứ tự
all_files = sorted(glob.glob(os.path.join(output_dir, '*.npy')))

# Tải từng file batch và cho vào list
all_batches = []
for f in tqdm(all_files, desc="Đang tải các batch đã lưu"):
    all_batches.append(np.load(f))

# Ghép lại thành 1 ma trận numpy lớn
test_embeddings = np.concatenate(all_batches, axis=0)

print("--- KIỂM TRA SHAPE ---")
print(f"Số lượng protein trong test_sequences_df: {len(test_sequences_df)}")
print(f"Shape của ma trận test embeddings cuối cùng: {test_embeddings.shape}")
# (Hai con số đầu tiên phải khớp nhau)

# *** PHẦN SỬA QUAN TRỌNG: LƯU FILE CUỐI VÀO DRIVE ***
final_test_embedding_file = os.path.join(SAVE_PATH, 'test_embeddings_esm150M.npy')
np.save(final_test_embedding_file, test_embeddings)
print(f"Đã lưu file test embeddings tổng vào: {final_test_embedding_file}")

Thư mục lưu batch test: /content/drive/MyDrive/CAFA6_Project/test_embeddings_batches
Bắt đầu tạo embeddings cho 224309 protein test...


Tạo test embedding:   0%|          | 0/3505 [00:00<?, ?it/s]

Đã hoàn thành tạo tất cả các batch embeddings!
Đang ghép các file batch từ Google Drive...


Đang tải các batch đã lưu:   0%|          | 0/3505 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
model_file = os.path.join(SAVE_PATH, 'best_model.keras')
test_embedding_file = os.path.join(SAVE_PATH, 'test_embeddings_esm150M.npy')

print(f"Đang tải mô hình đã huấn luyện từ: {model_file}")
# Tải mô hình đã huấn luyện
model = tf.keras.models.load_model(model_file)

print(f"Đang tải test embeddings từ: {test_embedding_file}")
# Tải ma trận test embeddings
test_embeddings = np.load(test_embedding_file)

print(f"Shape của test embeddings: {test_embeddings.shape}")

Đang tải mô hình đã huấn luyện từ: /content/drive/MyDrive/CAFA6_Project/best_model.keras
Đang tải test embeddings từ: /content/drive/MyDrive/CAFA6_Project/test_embeddings_esm150M.npy
Shape của test embeddings: (224309, 640)


In [29]:
BATCH_SIZE = 64

In [30]:
print("Bắt đầu dự đoán (predict) trên tập test...")
# Đây là ma trận xác suất (shape: [num_test_proteins, num_go_terms])
test_predictions = model.predict(
    test_embeddings,
    batch_size=BATCH_SIZE * 4 # Có thể tăng batch_size khi predict để nhanh hơn
)

print(f"Shape của ma trận dự đoán: {test_predictions.shape}")

Bắt đầu dự đoán (predict) trên tập test...
877/877 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step
Shape của ma trận dự đoán: (224309, 26125)


In [39]:
import joblib
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

print("--- Bước 5.3 (Phiên bản TỐI ƯU RAM) ---")

# --- ĐẢM BẢO CÁC BIẾN NÀY ĐÃ TỒN TẠI ---
# 1. SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'
# 2. test_sequences_df (DataFrame chứa EntryID)
# 3. test_predictions (Ma trận numpy từ Bước 5.2)
# ----------------------------------------------------

# 1. Tải các tài nguyên cần thiết
print("Đang tải binarizer và lấy danh sách EntryID...")
binarizer_file = os.path.join(SAVE_PATH, 'multilabel_binarizer.joblib')
mlb = joblib.load(binarizer_file)
go_term_columns = mlb.classes_
test_entry_ids = test_sequences_df['EntryID'].tolist()

# 2. Định nghĩa file output và kích thước chunk
submission_file_path = 'submission.tsv'
CHUNK_SIZE = 5000 # Xử lý 5000 protein mỗi lần (bạn có thể tăng nếu RAM vẫn còn)
num_chunks = (len(test_predictions) // CHUNK_SIZE) + 1

print(f"Sẽ xử lý {len(test_predictions)} protein theo {num_chunks} chunks...")

# 3. Vòng lặp xử lý từng chunk
for i in tqdm(range(0, len(test_predictions), CHUNK_SIZE), desc="Xử lý và lưu chunks"):

    # Lấy ra một chunk (lô) dự đoán và ID
    start_idx = i
    end_idx = min(i + CHUNK_SIZE, len(test_predictions))

    preds_chunk = test_predictions[start_idx:end_idx]
    ids_chunk = test_entry_ids[start_idx:end_idx]

    # Tạo DataFrame "wide" CHỈ CHO CHUNK NÀY (nhỏ, vừa RAM)
    chunk_df_wide = pd.DataFrame(preds_chunk, columns=go_term_columns)
    chunk_df_wide['EntryID'] = ids_chunk

    # Melt CHỈ CHUNK NÀY
    chunk_df_long = chunk_df_wide.melt(
        id_vars=['EntryID'],
        var_name='term',
        value_name='probability'
    )

    # Lọc bỏ xác suất thấp NGAY LẬP TỨC
    chunk_df_long = chunk_df_long[chunk_df_long['probability'] > 0.01]

    # Ghi vào file
    if i == 0:
        # Lần chạy đầu tiên, ghi đè file (write mode 'w') và KHÔNG có header
        chunk_df_long.to_csv(
            submission_file_path,
            sep='\t',
            index=False,
            header=False,
            mode='w'
        )
    else:
        # Các lần sau, ghi nối tiếp (append mode 'a') và KHÔNG có header
        chunk_df_long.to_csv(
            submission_file_path,
            sep='\t',
            index=False,
            header=False,
            mode='a'
        )

print("--- HOÀN TẤT TOÀN BỘ QUY TRÌNH! ---")
print(f"File '{submission_file_path}' đã được tạo thành công (tối ưu RAM).")

--- Bước 5.3 (Phiên bản TỐI ƯU RAM) ---
Đang tải binarizer và lấy danh sách EntryID...
Sẽ xử lý 224309 protein theo 45 chunks...


Xử lý và lưu chunks:   0%|          | 0/45 [00:00<?, ?it/s]

--- HOÀN TẤT TOÀN BỘ QUY TRÌNH! ---
File 'submission.tsv' đã được tạo thành công (tối ưu RAM).


**Result: 0.258 -> Rank 207**

## Improvement

### Post-processing

In [31]:
# Cài đặt thư viện networkx để xử lý đồ thị
!pip install networkx

import networkx as nx
import pandas as pd
import joblib
import numpy as np
from tqdm.auto import tqdm
import os

# --- ĐẢM BẢO CÁC BIẾN/FILE NÀY ĐÃ TỒN TẠI ---
# 1. Biến SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'
# 2. File 'multilabel_binarizer.joblib' (trên Drive)
# 3. DataFrame 'nodes_df' (từ code gốc của bạn)
# 4. Đảm bảo bạn đã KẾT NỐI GOOGLE DRIVE
# ----------------------------------------------------

print("Đang tải MultiLabelBinarizer (mlb) từ Drive...")
binarizer_file = os.path.join(SAVE_PATH, 'multilabel_binarizer.joblib')
mlb = joblib.load(binarizer_file)

# Tạo map (ánh xạ) từ GO_ID -> index (cột) trong ma trận dự đoán
# Đây là bước cực kỳ quan trọng
go_to_index = {go_id: idx for idx, go_id in enumerate(mlb.classes_)}

print(f"Đã tải {len(mlb.classes_)} GO terms từ binarizer.")
# (Bạn cần đảm bảo 'nodes_df' đã được tải vào bộ nhớ từ code gốc)
# Giả sử 'nodes_df' đã tồn tại...

Đang tải MultiLabelBinarizer (mlb) từ Drive...
Đã tải 26125 GO terms từ binarizer.


### Construct GO Graph

In [35]:
print("Đang xây dựng Đồ thị GO từ nodes_df...")

# 1. Xây dựng đồ thị (parent -> child)
go_graph = nx.DiGraph()

# 2. Xây dựng map (child -> [parents])
child_to_parents_map = {}

# Giả sử nodes_df của bạn có cột 'id' và 'is_a'
for _, row in tqdm(nodes_df.iterrows(), total=len(nodes_df), desc="Xử lý nodes_df"):
    child_id = row['id']

    # Xử lý cột 'is_a' (có thể là NaN hoặc list)
    if isinstance(row['is_a'], list):
        # Tách lấy ID của cha
        parents = [p.split(' ! ')[0] for p in row['is_a']]

        # Lưu vào map
        child_to_parents_map[child_id] = parents

        # Thêm các cạnh vào đồ thị
        for parent_id in parents:
            go_graph.add_edge(parent_id, child_id)

print("Đã xây dựng xong đồ thị!")

Đang xây dựng Đồ thị GO từ nodes_df...


Xử lý nodes_df:   0%|          | 0/40122 [00:00<?, ?it/s]

Đã xây dựng xong đồ thị!


### Topological sort

In [36]:
# Sắp xếp Topo cho chúng ta thứ tự [gốc, ..., lá]
# Chúng ta cần đảo ngược lại (reversed) để đi từ [lá, ..., gốc]
try:
    reversed_topological_sort = list(reversed(list(nx.topological_sort(go_graph))))
    print(f"Tổng số node trong thứ tự sắp xếp: {len(reversed_topological_sort)}")
except nx.NetworkXUnfeasible:
    print("LỖI: Đồ thị có chu trình (cycle), không thể sắp xếp topo.")
    # (Nếu lỗi này xảy ra, chúng ta cần xử lý chu trình,
    # nhưng với GO-basic thì hiếm khi)

Tổng số node trong thứ tự sắp xếp: 40122


In [37]:
print(f"Shape của 'test_predictions' (trước khi hiệu chỉnh): {test_predictions.shape}")

Shape của 'test_predictions' (trước khi hiệu chỉnh): (224309, 26125)


### Propagation

In [38]:
print("Bắt đầu hiệu chỉnh xác suất (propagation)...")

# Chúng ta lặp qua thứ tự đã sắp xếp (từ lá -> gốc)
for child_go_id in tqdm(reversed_topological_sort, desc="Propagating Probs"):

    # Kiểm tra xem "con" này có trong map cha và có trong mô hình của chúng ta không
    if child_go_id in child_to_parents_map and child_go_id in go_to_index:

        # Lấy index (cột) và vector xác suất của 'con'
        child_idx = go_to_index[child_go_id]
        p_child = test_predictions[:, child_idx]

        # Lấy danh sách 'cha' của nó
        for parent_go_id in child_to_parents_map[child_go_id]:

            # Kiểm tra xem 'cha' này có trong mô hình của chúng ta không
            if parent_go_id in go_to_index:

                # Lấy index (cột) và vector xác suất của 'cha'
                parent_idx = go_to_index[parent_go_id]
                p_parent = test_predictions[:, parent_idx]

                # Áp dụng quy tắc: P(cha) = max(P(cha), P(con))
                # Sử dụng np.maximum cho MỌI protein (toàn bộ vector) cùng lúc
                test_predictions[:, parent_idx] = np.maximum(p_parent, p_child)

print("--- HOÀN THÀNH HIỆU CHỈNH! ---")

Bắt đầu hiệu chỉnh xác suất (propagation)...


Propagating Probs:   0%|          | 0/40122 [00:00<?, ?it/s]

--- HOÀN THÀNH HIỆU CHỈNH! ---


Rerun the submission file cell

**Score: 0.223 -> Decrease.**

## Improvement 2

In [40]:
import pandas as pd
import numpy as np
import joblib
import networkx as nx
from tqdm.auto import tqdm
import os

# --- Đảm bảo các biến/file này đã tồn tại ---
# 1. Biến SAVE_PATH = '/content/drive/MyDrive/CAFA6_Project/'
# 2. DataFrame 'nodes_df' (từ code gốc)
# 3. DataFrame 'terms_df' (từ code gốc)
# 4. DataFrame 'sequences_df' (từ code gốc, để lấy thứ tự)
# 5. Đảm bảo bạn đã KẾT NỐI GOOGLE DRIVE
# ----------------------------------------------------

print("Đang tải lại nodes_df và terms_df...")
# (Bạn hãy chạy lại code tải 2 file này nếu chúng không còn trong bộ nhớ)

Đang tải lại nodes_df và terms_df...


### Construct ancestor map

In [42]:
print("Đang xây dựng Đồ thị GO...")
go_graph = nx.DiGraph()
for _, row in tqdm(nodes_df.iterrows(), total=len(nodes_df), desc="Xử lý nodes_df"):
    child_id = row['id']
    if isinstance(row['is_a'], list):
        parents = [p.split(' ! ')[0] for p in row['is_a']]
        for parent_id in parents:
            # Chú ý: Cạnh đồ thị đi từ CON -> CHA
            # để dễ dàng tìm tổ tiên
            go_graph.add_edge(child_id, parent_id)

print("Đã xây dựng xong đồ thị!")

# Tạo map tổ tiên
print("Đang tạo map tổ tiên (sẽ mất vài phút)...")
# nx.ancestors(G, node) trả về TẤT CẢ các node có thể đi đến từ 'node'
# (Vì chúng ta xây dựng cạnh con->cha, nên đây chính là tổ tiên)
ancestor_map = {node: nx.ancestors(go_graph, node) for node in tqdm(go_graph.nodes(), desc="Tìm tổ tiên")}
print("Đã tạo xong map tổ tiên.")

Đang xây dựng Đồ thị GO...


Xử lý nodes_df:   0%|          | 0/40122 [00:00<?, ?it/s]

Đã xây dựng xong đồ thị!
Đang tạo map tổ tiên (sẽ mất vài phút)...


Tìm tổ tiên:   0%|          | 0/40122 [00:00<?, ?it/s]

Đã tạo xong map tổ tiên.


In [49]:
from sklearn.preprocessing import MultiLabelBinarizer

print("--- Bước 3 (MỚI): Mở rộng Labels ---")

# 1. Lấy danh sách labels gốc (như Bước 3 cũ)
print("Đang nhóm các term gốc...")
labels_grouped_df = terms_df.groupby('EntryID')['term'].apply(list).reset_index()

# 2. Mở rộng (Propagate) các nhãn
print("Đang truyền bá (propagate) nhãn lên tổ tiên...")

def expand_labels(original_terms_list):
    # Dùng set để tránh trùng lặp
    expanded_set = set(original_terms_list)

    for term in original_terms_list:
        if term in ancestor_map:
            # Thêm tất cả tổ tiên của term này vào set
            expanded_set.update(ancestor_map[term])

    return list(expanded_set)

# Áp dụng hàm mở rộng cho mọi protein (có thể mất 1-2 phút)
labels_grouped_df['expanded_terms'] = labels_grouped_df['term'].apply(expand_labels)

# 3. Lấy danh sách TẤT CẢ các GO term duy nhất (sau khi mở rộng)
# Điều này rất quan trọng
all_expanded_terms_set = set()
for term_list in labels_grouped_df['expanded_terms']:
    all_expanded_terms_set.update(term_list)
all_terms_sorted = sorted(list(all_expanded_terms_set))
num_classes = len(all_terms_sorted)
print(f"Tổng số lớp (GO term) duy nhất SAU KHI MỞ RỘNG: {num_classes}")


# 4. Căn chỉnh (Align) với `sequences_df` (giống Bước 3 cũ)
print("Đang căn chỉnh thứ tự labels...")
ordered_entry_ids_df = sequences_df[['EntryID']]
aligned_labels_df = ordered_entry_ids_df.merge(labels_grouped_df, on='EntryID', how='left')

# Dùng 'expanded_terms' thay vì 'term'
aligned_labels_df['expanded_terms'] = aligned_labels_df['expanded_terms'].apply(lambda x: x if isinstance(x, list) else [])

# 5. Binarize
print("Đang binarize các labels đã mở rộng...")
mlb_expanded = MultiLabelBinarizer(classes=all_terms_sorted)
labels_list_expanded = aligned_labels_df['expanded_terms'].tolist()
full_labels_matrix_expanded = mlb_expanded.fit_transform(labels_list_expanded)

# 6. Lưu lại (File MỚI)
print("Đang lưu labels và binarizer MỚI vào Google Drive...")
np.save(SAVE_PATH + 'train_labels_EXPANDED.npy', full_labels_matrix_expanded)
joblib.dump(mlb_expanded, SAVE_PATH + 'multilabel_binarizer_EXPANDED.joblib')

print("--- HOÀN THÀNH BƯỚC 3 (MỚI)! ---")

--- Bước 3 (MỚI): Mở rộng Labels ---
Đang nhóm các term gốc...
Đang truyền bá (propagate) nhãn lên tổ tiên...
Tổng số lớp (GO term) duy nhất SAU KHI MỞ RỘNG: 39514
Đang căn chỉnh thứ tự labels...
Đang binarize các labels đã mở rộng...
Đang lưu labels và binarizer MỚI vào Google Drive...
--- HOÀN THÀNH BƯỚC 3 (MỚI)! ---
